# NONNEGATIVE MATRIX FACTORISATION FOR TOPIC EXTRACTION
## - TOPIC EXTRACTION FROM DOCUMENTS -

The goal is to study the use of nonnegative matrix factorisation (NMF) for topic extraction from a dataset of text documents. The rationale is to interpret each extracted NMF component as being associated with a specific topic.

In [14]:
# -*- coding: utf-8 -*-
# Author: Olivier Grisel <olivier.grisel@ensta.org>
# Lars Buitinck
# Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause
from __future__ import print_function
from time import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups

In [4]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]\
        for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [5]:
# Load the 20 newsgroups dataset and vectorize it. We use a few
# heuristics to filter out useless terms early on: the posts are
# stripped of headers, footers and quoted replies, and common
# English words, words occurring in only one document or in at
# least 95% of the documents are removed.
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 161.381s.


In [6]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
max_features=n_features,
stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features...
done in 1.289s.


In [7]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
"n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 5.391s.


In [8]:
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood
Topic #6: file problem files format win sound ftp pub read save site help image available create copy running memory self version
Topic #7: game team games year win play season playe

In [ ]:
def custom_NMF(V, K, W=None, H=None, steps=50, beta=0, toll=0.1, show_div=False):
    
    def _beta_div(V,W,H,beta,F,N,K):
    div = 0
    # Update beta_divergence
    if beta == 1: # generalized Kullback-Leibler divergence. x log(x/y) - x + y
        # div = numpy.dot(V, numpy.log(V,numpy.dot(W,H))) - numpy.sum(V) + numpy.sum(numpy.dot(W,H))
        func = _kullback_leiber
    elif beta == 0: # Itakura-Saito divergence. (x/y) - log(x/y) -1
        # div = numpy.sum(V / numpy.dot(W,H)) - numpy.sum(numpy.log(V / numpy.dot(W,H))) - numpy.product(len(V))
        func = _itakura_saito
    else: # Euclidean distance. (1/beta(beta-1))(x^beta + (beta-1)y^beta - beta*x*y^beta-1)
        func = _euclidean_distance
    WH = numpy.dot(W, H)
    for i in range(F):
        for j in range(N):
            x = V[i][j]
            if x == 0:
                x = numpy.finfo(numpy.double).tiny
            y = WH[i][j]
            div += func(x,y,beta)
    return div

    def _kullback_leiber(x,y,beta):
        return x*numpy.log(x/y) - x + y

    def _itakura_saito(x,y,beta):
        return x*numpy.log(x/y) - x + y

    def _euclidean_distance(x,y,beta):
        return (1/(beta*(beta-1)))*(pow(x,beta) + (beta-1)*pow(y,beta) - beta*x*pow(y,beta-1))

    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = numpy.random.rand(F,K)
        
    if H is None:
        H = numpy.random.rand(K,N)
        
    if N != len(H[0]):
        raise ValueError("Size for H[0] is different - found "+str(len(H[0]))+" in place of "+str(N))
    if F != len(W):
        raise ValueError("Size for F is different - found "+str(len(F))+" in place of "+str(N))
        
    #Setup n_iter
    n_iter = 1
    
    # Setup initial error
    init_error = _beta_div(V,W,H,beta,F,N,K)
    if show_div:
        print("Initial error: "+str(init_error))
    error = init_error
    
    for step in range(steps):
    
        # Tests with whole matrix : multiply = O | dot = *
        upd_UP = numpy.dot(W.T, numpy.multiply(pow(numpy.dot(W,H),beta-2), V))
        upd_DOWN = numpy.dot(W.T, pow(numpy.dot(W,H),beta-1))
        upd = upd_UP / upd_DOWN
        H = numpy.multiply(H, upd)
        
        upd_UP = numpy.dot(numpy.multiply(pow(numpy.dot(W,H),beta-2), V),H.T)
        upd_DOWN = numpy.dot(pow(numpy.dot(W,H),beta-1), H.T)
        upd = upd_UP / upd_DOWN
        W = numpy.multiply(W, upd)

        if toll > 0:
            new_error = _beta_div(V,W,H,beta,F,N,K)
            if show_div:
                print("Error on iteration "+str(n_iter)+": " +str(new_error))
            # Check if approximation error relative decrease is below the desired threshold
            rel_dec = ((error - new_error) / init_error)
            if show_div:
                print("Error relative decrease at iteration "+str(n_iter)+": "+str(rel_dec))
            if rel_dec < toll:
                break
            error = new_error
            
        n_iter += 1
            
    return W, H

### 1. Test and comment on the effect of varying the initialisation, especially using random nonnegative values as initial guesses (for W and H coefficients, using the notations introduced during the lecture).

In [9]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
"n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1, 
          init='random', alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.853s.

Topics in NMF model (Frobenius norm):
Topic #0: just like don people time know good make way use right say ve really want government ll new did going
Topic #1: christian bible true christians faith jesus religion people does christ belief church life truth read reading believe statement atheism claim
Topic #2: god jesus sin heaven lord christ believe mary does bible love human knowledge life marriage faith say children atheism knows
Topic #3: think don people win extra just early sold sex need actually happen means pretty toronto wasn agree david statement mike
Topic #4: drive drives hard disk software floppy card mac 00 computer scsi controller power apple mb pc sale rom monitor internal
Topic #5: thanks know does mail advance hi info interested email anybody looking card help like appreciated list send information video need
Topic #6: windows file dos files program usin

### 2. Compare and comment on the difference between the results obtained with `2 cost compared to the generalised Kullback-Liebler cost.

### 3. Test and comment on the results obtained using a simpler term-frequency representation as input (as opposed to the TF-IDF representation considered in the code above) when considering the Kullback-Liebler cost.

## - CUSTOM NMF IMPLEMENTATION -
Implement the multiplicative update rules (derived from the majorisation-minimisation approach) for NMF estimation with β divergences, including the case β = 1 (generalised Kullback-Liebler divergence). Ensure that:
- You can easily choose a custom initialisation for the W and H matrices;
- You can set a custom number of iteration;
- You can monitor the behaviour of the loss function across the iterations and that it is readily decreasing.

Compare your implementation with the one offered by scikit-learn.

In [31]:
def nfm(V=None, W=None, H=None, k=2, max_iter=1000, alpha=0.0002, beta=1):
    
    # If V is not passed, we can't really do anything
    if V is None:
        raise ValueError('Please define matrix V')
     
    # If W is not passed, initialize as random
    if W is None:
        W = np.random.rand(V.shape[0], k)

    # If H is not passed, initialize as random
    if H is None:
        H = np.random.rand(k, V.shape[1])
        
    # V and W must have same number of rows
    if V.shape[0] != W.shape[0]:
        raise ValueError('V and W must have same number of rows')
        
    # V and H must have same number of columns
    if H.shape[1] != H.shape[1]:
        raise ValueError('V and H must have same number of columns')
    
    # W and H must have respectively columns and rows equal to k
    if (W.shape[1] != k) | (H.shape[0] != k) :
        raise ValueError('W and H must have respectively columns and rows equal to k.')
    
    r = V.shape[0]
    c = V.shape[1]
    for step in range(max_iter):
#         for i in range(r):
#             for j in range(c):
#                 if V[i][j] > 0:
#                     eij = V[i][j] - np.dot(W[i,:], H[:,j])
#                     for k in range(k):
# #                         W[i][k] = W[i][k] + alpha * (2 * eij * H[k][j] - beta * W[i][k])
# #                         H[k][j] = H[k][j] + alpha * (2 * eij * W[i][k] - beta * H[k][j])

#                         H[k][j] = H[k][j]*((W[])) + alpha * (2 * eij * W[i][k] - beta * H[k][j])
#                         W[i][k] = W[i][k]*((W[i][k]*H[i][k])) + alpha * (2 * eij * H[k][j] - beta * W[i][k])
        H_upper = np.dot(W.T, np.multiply(np.power(np.dot(W, H), (beta-2)), V))
        H_lower = np.dot(W.T, np.power(np.dot(W, H), (beta-1)))
        H = np.multiply(H, np.divide(H_upper, H_lower))
        
        W_upper = np.dot(np.multiply(np.power(np.dot(W, H), (beta-2)), V), H.T)
        W_lower = np.dot(np.power(np.dot(W, H), (beta-1)), H.T)
        W = np.multiply(W, np.divide(W_upper, W_lower))
        
        bdiv = 0
        
        eV = np.dot(W, H)
        e = 0
        for i in range(r):
            for j in range(c):
                if V[i][j] > 0:
                    e = e + pow(V[i][j] - np.dot(W[i,:], H[:,j]), 2)
                    for k in range(k):
                        e = e + (beta/2) * ( pow(W[i][k], 2) + pow(H[k][j], 2) )
        if e < 0.001:
            break
    return W, H.T

In [32]:
R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
]
R = np.array(R)

N = len(R)
M = len(R[0])
K = 2

P = np.random.rand(N,K)
Q = np.random.rand(K, M)

nP, nQ = nfm(V=R, W=P, H=Q, k=K)
nP

C:\Users\Utente\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in power
C:\Users\Utente\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in multiply


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [ ]:
import numpy as np
from scipy import linalg
from numpy import dot

def nmf(X, latent_features, max_iter=100, error_limit=1e-6, fit_error_limit=1e-6):
    """
    Decompose X to A*Y
    """
    eps = 1e-5
    print 'Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter)
    X = X.toarray()  # I am passing in a scipy sparse matrix

    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # ===== updates =====
        # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # ==== evaluation ====
        if i % 5 == 0 or i == 1 or i == max_iter:
            print 'Iteration {}:'.format(i),
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

            curRes = linalg.norm(mask * (X - X_est), ord='fro')
            print 'fit residual', np.round(fit_residual, 4),
            print 'total residual', np.round(curRes, 4)
            if curRes < error_limit or fit_residual < fit_error_limit:
                break

return A, Y